# SQL. Запросы к базе данных. 

В базе данных одного из конкурирующих сервисов есть информация о книгах, издательствах, авторах, а также пользовательские обзоры на книги. Эти данные помогут сформулировать ценностное предложение для нового продукта.

<b>Задачи:</b>

- Посчитайте количество книг, выпущенных после 1 января 2000 года;
- Посчитайте количество пользовательских обзоров и среднюю оценку для каждой книги;
- Определите издательство, которое издало наибольшее число книг толще 50 страниц (так вы исключите из анализа различные брошюры);
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более пользовательскими оценками;
- Посчитайте среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
#сделаем функцию, чтобы было легче читать таблички.
def SQL (query): 
    return pd.io.sql.read_sql(query, con = engine)

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

In [3]:
SQL('''SELECT * FROM Books LIMIT 5''')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

In [4]:
SQL('''SELECT * FROM authors LIMIT 5''')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

In [5]:
SQL('''SELECT * FROM publishers LIMIT 5''')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

In [6]:
SQL('''SELECT * FROM ratings LIMIT 5''') 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [7]:
SQL('''SELECT * FROM reviews LIMIT 5''') 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Количество книг, выпущенных после 1 января 2000 года

In [8]:
SQL('''SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01';''')

,count
0,819


С 1 января 2000 года было выпущего 819 книг.


### Количество пользовательских обзоров и среднюю оценку для каждой книги

In [9]:
SQL('''SELECT ratings.book_id, COUNT(DISTINCT reviews.review_id) AS cnt_review, AVG(ratings.rating) AS avg_rating
FROM ratings
FULL OUTER JOIN reviews
ON ratings.book_id = reviews.book_id
GROUP BY ratings.book_id
LIMIT 15;''')

,book_id,cnt_review,avg_rating
0,1,2,3.666667
1,2,1,2.500000
2,3,3,4.666667
3,4,2,4.500000
4,5,4,4.000000
5,6,4,3.500000
6,7,3,3.000000
7,8,3,4.000000
8,9,2,4.250000
9,10,3,3.200000


### Определить издательство, которое издало наибольшее число книг толще 50 страниц

In [10]:
SQL('''SELECT books.publisher_id, publisher,  COUNT(book_id)
FROM books
FULL OUTER JOIN publishers
ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publisher, books.publisher_id
ORDER BY count DESC
LIMIT 10;''')

,publisher_id,publisher,count
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,33,Ballantine Books,19
5,35,Bantam,19
6,45,Berkley,17
7,284,St. Martin's Press,14
8,46,Berkley Books,14
9,83,Delta,13


Penguin Books - издало 42 книги толщиной больше 50 страниц. Из данной выборки, это наибольшее кол-во книг среди прочих издательств по заданным меркам. 

### Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более пользовательскими оценками

In [11]:
SQL('''SELECT authors.author AS author, AVG(cnt_rating.avg) as very_tired
FROM authors
FULL OUTER JOIN books
ON books.author_id = authors.author_id 
FULL OUTER JOIN (SELECT book_id, COUNT(rating) as cnt, AVG(rating) as avg
           FROM ratings
           GROUP BY book_id) AS cnt_rating
           ON cnt_rating.book_id = books.book_id 
           WHERE cnt >= 50
GROUP BY author
ORDER BY 
 very_tired DESC
;''')

,author,very_tired
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Кто бы мог подумать, что Поулинг окажется в топе. Сюрприз-сюрприз :) 

### Посчитайте среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок.

In [12]:
SQL('''SELECT ROUND(AVG(count))
    FROM
    (SELECT COUNT(text)
FROM reviews
FULL OUTER JOIN (SELECT username, COUNT(rating_id) as cnt
           FROM ratings
           GROUP BY username) AS cnt_rating
ON cnt_rating.username = reviews.username
WHERE cnt > 50
GROUP BY reviews.username) AS Its_very_hard;''')

,round
0,24.0


Среднее кол-во текстовых обзоров пользователей, которые поставили более чем 50 оценок = 23 текстовых обзора в среднем. 